In [ ]:
import plotly.express as px

from random import randrange
import numpy as np
import pandas as pd

import plotly.io as pio
import plotly.graph_objects as go
from pathlib import Path

import plotly.io as pio
# pio.kaleido.scope.mathjax = None # Prevent text box in generated pdf https://github.com/plotly/plotly.py/issues/3469
out_folder = "figures"


def remove_labels_equals_part(fig):
    def safeSplit(name):
        names = name.split('=', 1)
        return names[1] if len(names) > 1 else names[0]

    fig.for_each_trace(
        lambda t: t.update(
            name=safeSplit(t.name),
            legendgroup=safeSplit(t.name),
            hovertemplate=t.hovertemplate.replace(t.name, safeSplit(t.name))))
    fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", " ")))


def remove_labels_column_name(fig, if_startswith=None):
    def safeSplit(name):
        if if_startswith is not None and name.startswith(if_startswith):
            names = name.split('=', 1)
            return names[1] if len(names) > 1 else names[0]
        else:
            return name

    fig.for_each_trace(
        lambda t: t.update(
            name=safeSplit(t.name),
            legendgroup=safeSplit(t.name),
            hovertemplate=t.hovertemplate.replace(t.name, safeSplit(t.name))))
    fig.for_each_annotation(lambda a: a.update(text=safeSplit(a.text)))


pio.templates["garamond"] = go.layout.Template(
    {'layout': {'font': {'family': 'Garamond'}, 'colorway': px.colors.qualitative.T10}})
pio.templates.default = "plotly_white+garamond"
category_orders_cifar = {
    "Layer":
    ["conv2d", "max_pooling2d", "conv2d_1", "max_pooling2d_1", "conv2d_2", "flatten", "dense",
     "dense_1"],
    "Input size": [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]}
category_orders_mnist = {"Layer": ["conv2d", "max_pooling2d",
                                   "conv2d_1", "max_pooling2d_1", "flatten", "dropout", "dense"], }
category_orders_inception_v3 = {"Layer": [f"mixed{i}" for i in range(1, 11)] + ["prediction"]}
category_orders_resnet = {"Layer": ["pool1_pool", "conv2_block3_out",
                                    "conv3_block4_out", "conv4_block6_out", "conv5_block3_out", "prediction"]}

category_orders = {"mnist": category_orders_mnist, "cifar10": category_orders_cifar,
                   "inception_v3": category_orders_inception_v3,
                   "resnet50": category_orders_resnet, "resnet50v2": category_orders_resnet}


def read_data(path='model_pattern_count.feather'):

    df = pd.read_feather(path)
    df["Layer aggregation"].replace(
        {"MaxAggregation": "Peak feature strength", "MeanAggregation": "Feature amount",
         "MinMaxAggregation": "Feature range", "MeanStdAggregation": "Feature amount+spread",
         "NoAggregation": "None"},
        inplace=True)
    return df


In [ ]:
def aggregation_method(df, models, layer=None):

    dff = df[(df["Model"].isin(models)) & (df["Minimum pattern size"] == 5) & (df["Input size"] == 12811) &
             (df["Layer"] != "predictions") &  # predictions layer is not convolutional
             (df["Epsilon"] == 0) & (df["Split"] == "validation")]
    if layer is not None:
        dff = dff[dff["Layer"] == layer]
    cat_orders = category_orders.copy()
    cat_orders["Layer aggregation"] = ["Peak feature strength",
                                       "Feature range", "Feature amount", "Feature amount+spread", "None"]
    fig = px.bar(dff,
                 x="Layer aggregation", y="Number of patterns",
                 facet_col="Layer",
                 color="Model" if len(models) > 1 else "Layer aggregation",
                 barmode='group' if len(models) > 1 else 'stack',
                 # height=600,
                 category_orders=cat_orders,
                 title=f"Model: {models}")
    remove_labels_column_name(fig, "Layer")
    if layer is not None:
        out_name = f"{models}_{layer}_aggregation.pdf"
    else:
        out_name = f"{models}_aggregation.pdf"
    #fig.write_image(Path(out_folder, out_name))
    return fig


df = read_data()
print(df["Model"].unique())
aggregation_method(df, ["inception_v3"], "mixed10").show()
aggregation_method(df, ["resnet50", "resnet50v2"], "conv5_block3_out").show()
aggregation_method(df, ["resnet50v2"], "conv5_block3_out").show()
aggregation_method(df, ["inception_v3"]).show()
aggregation_method(df, ["resnet50"]).show()
aggregation_method(df, ["resnet50v2"]).show()


In [ ]:
def input_size(df, models, layer=None, splits=None, title=None):

    dff = df[(df["Model"].isin(models)) & (df["Minimum pattern size"] == 5) &
             (df["Layer aggregation"] == "Feature amount") & (df["Epsilon"] == 0) & df["Split"].isin(
                 ["test", "validation"])]
    if layer is not None:
        dff = dff[dff["Layer"] == layer]
    if splits is not None:
        dff = dff[dff["Split"].isin(splits)]

    color = None
    if splits is not None and len(splits) > 1:

        color = "Split"
    elif len(models) > 1:
        color = "Model"
    cat_orders = category_orders.copy()
    cat_orders["Layer aggregation"] = ["Peak feature strength",
                                       "Feature range", "Feature amount", "Feature amount+spread", "None"]
    fig = px.bar(dff,
                 x="Input size", y="Number of patterns",
                 facet_col="Layer" if layer is None else None,
                 color=color,
                 barmode='group',
                 category_orders=cat_orders,
                 color_discrete_sequence=px.colors.qualitative.Plotly,
                 title=title)
    #remove_labels_column_name(fig, "Layer")
    if layer is not None:
        out_name = f"{models}_{layer}_input_size.pdf"
    else:
        out_name = f"{models}_input_size.pdf"
    #fig.write_image(Path(out_folder, out_name))
    return fig


df = read_data()
#input_size(df, ["cifar10"], "dense").show()
input_size(df, ["inception_v3"], "mixed10", ["validation"]).show()
input_size(df, ["resnet50", "resnet50v2"], "conv5_block3_out", ["validation"]).show()


In [ ]:
def input_size_vs_epsilon(df, model, layer=None):
    dff = df[(df["Model"] == model) & (df["Minimum pattern size"] == 5) & (df["Layer"] != "predictions") &
             (df["Layer aggregation"] == "Feature amount") & (df["Split"] == "validation")]
    dff["Epsilon"] = dff["Epsilon"].astype("category")
    cat_orders = category_orders.copy()
    if layer is not None:
        dff = dff[dff["Layer"] == layer]
    #cat_orders["inception_v3"]["Layer"] = [f"mixed{i}" for i in range(1, 11)]
    fig = px.bar(dff,
                 x="Input size", y="Number of patterns",
                 facet_col="Layer" if layer is None else None,
                 color="Epsilon",
                 barmode="group",
                 # height=600,
                 category_orders=cat_orders,
                 title=f"Model: {model}")
    #fig.write_image(Path(out_folder, f"{model}_aggregation.pdf"))
    return fig


df = read_data()
input_size_vs_epsilon(df, "inception_v3", "mixed10").show()
input_size_vs_epsilon(df, "resnet50", "conv5_block3_out").show()


### Artificial white background VS white

In [ ]:

df = pd.DataFrame({
    "Spatial disentaglement":
    ["Peak feature strength", "Feature amount", "Feature amount", "Feature amount",
     "Feature amount+spread", "Feature range"] +
    ["Peak feature strength", "Feature amount", "Feature amount", "Feature amount",
     "Feature amount+spread", "Feature range"],
    "Image type": ["Artificial white background" for i in range(0, 6)] +
    ["White background" for i in range(0, 6)],
    "Number of images": [50, 11, 6, 5, 58, 25] + [45, 0, 0, 0, 1, 21],
    "Pattern": [1, 0, 1, 2, 1, 1] + [1, 0, 1, 2, 1, 1], })
fig = px.bar(df, x="Spatial disentaglement", y="Number of images", barmode='stack',
             color="Image type", color_discrete_sequence=["rgb(223,223,223)", "rgb(143,143,143)"],
             category_orders={"Image type": ["Artificial white background", "White background"]})
fig.show()
